In [ ]:
# Standard Library Imports
import copy
import os
import sys
import traceback

# Third-Party Imports
import matplotlib.pyplot as plt
# import requests
# import ROOT
# import tqdm
# import uproot
import awkward as ak
import vector
vector.register_awkward()
# from cernopendata_client import searcher

# Local Application Imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from parse_atlas import consts, parser, schemas

%load_ext autoreload
%autoreload 2

In [ ]:
atlas_parser = parser.ATLAS_Parser()
atlas_parser.get_records_file_index([consts.ATLAS_ELECTROWEAK_BOSON], file_idx=[consts.MC_ZMUMU_ELECTROWEAK])
# atlas_parser.get_records_file_index([consts.ATLAS_MC_TOP_NOMINAL])

In [ ]:
atlas_parser.parse_indexed_files(schemas.E_POSI_MUON_SCHEMA, limit = 1)

In [ ]:
# atlas_parser.save_events()
atlas_parser.load_events_from_file(consts.ATLAS_ELECTROWEAK_BOSON_DATA)

In [ ]:
def selected_electrons(el):
    return el[(el.pt > 30 * consts.GeV) & (abs(el.eta) < 2.47)]

def selected_muons(mu):
    return mu[(mu.pt > 30 * consts.GeV) & (abs(mu.eta) < 2.47)]

def selected_jets(j):
    return j[(j.pt > 30 * consts.GeV) & (abs(j.eta) < 2.47)]

def no_overlap(obj1, obj2, deltaR=0.4):
    obj1, obj2 = ak.unzip(ak.cartesian([obj1, obj2], nested=True))
    return ak.all(obj1.deltaR(obj2) > deltaR, axis=-1)

def mjjj(jets):
    candidates = ak.combinations(jets, 3)
    j1, j2, j3 = ak.unzip(candidates)
    has_b = (j1.is_bjet + j2.is_bjet + j3.is_bjet) > 0
    candidates["p4"] = j1 + j2 + j3
    candidates = candidates[has_b]
    candidates = candidates[ak.argmax(candidates.p4.pt, axis=1, keepdims=True)]
    return candidates.p4.mass

def processed(events):
    events = copy.copy(events) # shallow copy
    events["Jets", "btag_prob"] = events.BTagging_AntiKt4EMPFlow.DL1dv01_pb
    events["Electrons"] = selected_electrons(events.Electrons)
    events["Muons"] = selected_muons(events.Muons)
    events["Jets"] = selected_jets(events.Jets)
    events["Jets"] = events.Jets[no_overlap(events.Jets, events.Electrons)]
    events["Jets", "is_bjet"] = events.Jets.btag_prob > 0.85
    events = events[
        (ak.num(events.Jets) >= 4) # at least 4 jets
        & ((ak.num(events.Electrons) + ak.num(events.Muons)) == 1) # exactly one lepton
        & (ak.num(events.Jets[events.Jets.is_bjet]) >= 2) # at least two btagged jets with prob > 0.85
    ]
    return ak.to_packed(events)

events = processed(atlas_parser.events)
plt.hist(ak.flatten(mjjj(events.Jets) / consts.GeV, axis=None), bins=1000)
plt.xlabel("Reconstructed Top Quark Mass (GeV)")
plt.ylabel("Number of Events")
plt.title("Distribution of Reconstructed Top Quark Mass")
plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
plt.legend()
plt.show()

print('Total events:', f"{len(atlas_parser.events):,}")
print('Events after filtering:', f"{len(events):,}")
print('Total events:', f"{len(atlas_parser.events):,}")
print('Events after filtering:', f"{len(events):,}")

In [ ]:
def selected_muons(mu):
    return mu[(mu.pt > 30 * consts.GeV) & (abs(mu.eta) < 2.5)]

def selected_electrons(el):
    return el[(el.pt > 30 * consts.GeV) & (abs(el.eta) < 2.47)]

def electron_posi_muon_antimuon(events):
    events = copy.copy(events) # shallow copy
    events["Electrons"] = selected_electrons(events.Electrons)
    events["Muons"] = selected_muons(events.Muons)
    events["Electrons", "is_neg"] = events.Electrons.charge < 0
    events["Muons", "is_neg"] = events.Muons.charge < 0
    events = events[
        (ak.num(events.Electrons) == 2) & (ak.num(events.Muons) == 2)
        & (ak.num(events.Electrons[events.Electrons.is_neg]) == 1) & (ak.num(events.Muons[events.Muons.is_neg]) == 1)
    ]
    return ak.to_packed(events)

# Process the events
# atlas_parser.load_events_from_file(consts.ATLAS_ELECTROWEAK_BOSON_DATA)

events = electron_posi_muon_antimuon(atlas_parser.events)

def combine_four_momentaa(electrons, muons):
    electron_candidates = ak.combinations(electrons, 2)
    e1, e2 = ak.unzip(electron_candidates)
    electron_p4 = e1 + e2
    
    muon_candidates = ak.combinations(muons, 2)
    m1, m2 = ak.unzip(muon_candidates)
    muon_p4 = m1 + m2
    # Align the jagged arrays (broadcast to same structure)
    electron_p4, muon_p4 = ak.broadcast_arrays(electron_p4, muon_p4)

    # Element-wise sum while maintaining jagged structure
    combined_p4 = electron_p4 + muon_p4

    # Convert to (pt, phi, eta, mass) and return mass
    return combined_p4.to_ptphietamass().mass

def combine_four_momentaaa(electrons, muons):
    # Debug: Print input data
    print("Electrons:", electrons)
    print("Muons:", muons[0])

    # For events with two electrons, compute their invariant mass
    e_pairs = ak.combinations(electrons, 2)
    e1, e2 = ak.unzip(e_pairs)
    combined_e = e1 + e2
    print("Combined Electron Four-Momenta:", combined_e)
    print("Electron Pair Components (pt, phi, eta, mass):", combined_e.to_ptphietamass())
    e_mass = combined_e.to_ptphietamass().mass
    print("Electron Pair Masses:", e_mass)

    # For events with two muons, compute their invariant mass
    m_pairs = ak.combinations(muons, 2)
    m1, m2 = ak.unzip(m_pairs)
    combined_m = m1 + m2
    print("Combined Muon Four-Momenta:", combined_m)
    print("Muon Pair Components (pt, phi, eta, mass):", combined_m.to_ptphietamass())
    m_mass = combined_m.to_ptphietamass().mass
    print("Muon Pair Masses:", m_mass)  

    # Since events are pre-filtered, we can concatenate the masses directly
    # Events with electrons will have no muons, and vice versa
    combined_mass = ak.concatenate([e_mass, m_mass], axis=0)
    return combined_mass

def combine_four_momenta(electrons, muons):
    # Define particle masses (GeV)
    ELECTRON_MASS = 0.000511  # 0.511 MeV
    MUON_MASS = 0.1057        # 105.7 MeV

    # Convert electrons to Lorentz vectors with explicit mass
    electrons_vec = ak.zip(
        {
            "pt": electrons.pt,
            "eta": electrons.eta,
            "phi": electrons.phi,
            "mass": ELECTRON_MASS * ak.ones_like(electrons.pt)  # Broadcast mass to match shape
        },
        with_name="Momentum4D",  # Use vector's Momentum4D type
    )
    
    # Convert muons to Lorentz vectors with explicit mass
    muons_vec = ak.zip(
        {
            "pt": muons.pt,
            "eta": muons.eta,
            "phi": muons.phi,
            "mass": MUON_MASS * ak.ones_like(muons.pt)  # Broadcast mass to match shape
        },
        with_name="Momentum4D",
    )

    # Compute invariant masses for electron pairs
    e_pairs = ak.combinations(electrons_vec, 2)
    e1, e2 = ak.unzip(e_pairs)
    combined_e = e1 + e2
    e_mass = combined_e.mass

    # Compute invariant masses for muon pairs
    m_pairs = ak.combinations(muons_vec, 2)
    m1, m2 = ak.unzip(m_pairs)
    combined_m = m1 + m2
    m_mass = combined_m.mass

    # Combine results
    combined_mass = ak.concatenate([e_mass, m_mass], axis=0)
    print(electrons_vec)
    return combined_mass
print(events)
inv_mass = combine_four_momenta(events.Electrons, events.Muons)

In [ ]:
    def process(self, events):
        vector.register_awkward()
        # type of dataset being processed, provided via metadata (comes originally from fileset)
        dataset_category = events.metadata["dataset_name"]

        # apply a cut to events, based on lepton charge and lepton type
        events = events[lepton_filter(events.lep_charge, events.lep_typeid)]

        # construct lepton four-vectors
        leptons = ak.zip(
            {"pt": events.lep_pt,
             "eta": events.lep_eta,
             "phi": events.lep_phi,
             "energy": events.lep_energy},
            with_name="Momentum4D",
        )

        # calculate the 4-lepton invariant mass for each remaining event
        # this could also be an expensive calculation using external tools
        mllll = (
            leptons[:, 0] + leptons[:, 1] + leptons[:, 2] + leptons[:, 3]
        ).mass / 1000

In [ ]:
# Plot the invariant mass distribution

plt.hist(ak.flatten(inv_mass / consts.GeV), bins=100, range=(0, 500))
plt.xlabel("Invariant Mass (GeV)")
plt.ylabel("Number of Events")
plt.title("Distribution of Electron, Positron, Muon and Anti Muon")
plt.title("Distribution of Electron, Positron, Muon and Anti Muon")
plt.legend()
plt.show()

print('Total events:', f"{len(atlas_parser.events):,}")
print('Events after filtering:', f"{len(events):,}")